In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 10 10:18:55 2025

@author: adiazfl
Validated
"""
import sys
import os

# Add the source and examples directory to the path to run examples
#source_dir      = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
#sys.path.insert(0, source_dir)

# External libraries
import numpy as np
import sympy as sym

# This module libraries
import multibody as mb
from multibody import JointSystem, normalize_prismatic
from multibody import symVars_definition


In [ ]:
import m4e_attenuator as ex

MBDsys = mb.MBDSystem.from_example(ex)


Problem initialization finished in:		 0.002 seconds
All branches in the graph:
Branch 1: [1, 2]
R and RD matrices calculation finished in:	 0.081 seconds
Symbolic EOM computation finished in:		 0.005 seconds
Compiling EOM expressions finished in:		 0.067 seconds
['Acc', 'Acc_func', 'BDpoints_func', 'BDvel_func', 'BodyDataSym', 'Body_col', 'CGpoints_func', 'CGvel_func', 'Energy', 'Energy_func', 'Force', 'ForceAllCombined', 'Force_All', 'Force_func', 'ForcesPointsSym', 'GRpoints_func', 'Initial_Points', 'J', 'JOpoints_func', 'JointLoc', 'JointLoc_func', 'M', 'M_func', 'NDOF', 'Points_All', 'Pos', 'Pos_func', 'Q', 'QD', 'QDD', 'R', 'RD', 'RD_func', 'R_func', 'ReducedM', 'ReducedM_func', 'Reference_frame_Origin', 'Right_side', 'Right_side_func', 'Vel', 'Vel_All', 'Vel_func', '__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__

TypeError: animate_multibody() missing 3 required positional arguments: 'y', 'MBD', and 'mainNumVars'

In [7]:
print(MBDsys)
print(dir(MBDsys))

print(MBDsys.R.values)

print(MBDsys.ReducedM.values)

MBDSystem(joints=[[0, 1], [1, 2]], types=['R', 'R'], parent_cg_to_joint=[[0, 0], [-2, 0]], joint_to_child_cg=[[-1, 0], [-1, 0]], prismatic_direction=[Matrix([
[nan],
[nan]]), Matrix([
[nan],
[nan]])], Initial_Points={'GR': [], 'BD': {}}, Force={'PointsBD': [], 'CG': [], 'TensionSpring': [], 'TensionDamper': [], 'TorsionSpring': [], 'TorsionDamper': []}, Reference_frame_Origin=array([0, 0]), ForcesPointsSym=[], BodyDataSym=[], g=9.81, gVec=array([[1.],
       [1.]]), ic=array([0.1, 0. , 0. , 0. ]), joint_system=<multibody.MultibodyCore.joints_system.JointSystem object at 0x00000230265C8F10>, Q=[Theta_1, Theta_2], QD=[ThetaD_1, ThetaD_2], QDD=[ThetaDD_1, ThetaDD_2], NDOF=[1, 1], Body_col=[[0], [1]], BDpoints_func={}, BDvel_func={}, t_update=[])
['Acc', 'Acc_func', 'BDpoints_func', 'BDvel_func', 'BodyDataSym', 'Body_col', 'CGpoints_func', 'CGvel_func', 'Energy', 'Energy_func', 'Force', 'ForceAllCombined', 'Force_All', 'Force_func', 'ForcesPointsSym', 'GRpoints_func', 'Initial_Points', 'J'

In [4]:
# Do not modify 
Initial_Points  = {} 
Force           = {} 
# End of Do not Modify

# Example 4
Reference_frame_Origin = np.array([0,0.91])

# Bodies definition
dataNames   = [] # Define symbolic variables for force points (for example, b1, b2, b3).
BodyDataSym = symVars_definition(dataNames,globals()) # DO NOT MODIFY


joints              = [[0, 1],[1, 2],[1,3]]                                 # Joint connectivity: [parent, child]
types               = ['F','R', 'R']                                        # Joint types: 'R' for revolute, 'P' for prismatic, 'F' for floating
parent_cg_to_joint  = [[0, 0],[0.64, 0],[-0.64, 0]]                         # Vectors from parent's center-of-gravity (CG) to the joint location.
joint_to_child_cg   = [[np.nan, np.nan],[0, 0.17],[0, 0.17]]                # Vectors from the joint to the child's CG.
prismatic_direction = [[np.nan, np.nan],[np.nan, np.nan],[np.nan, np.nan]]  # For prismatic joints, the direction vector; for others, [nan, nan] is used.
prismatic_direction = normalize_prismatic(prismatic_direction)              # DO NOT MODIFY

# Points definition
PointNames  = [] # Define symbolic variables for force points (for example, b1, b2, b3).
PointsSym   = symVars_definition(PointNames,globals()) # DO NOT MODIFY

# Define the structure for initial points.
# Ground points: these are fixed and given as [x, z] coordinates.
Initial_Points["GR"]    = [[0.75, -0.91],[-0.75, -0.91]]

# Body-defined points ("BD"): we use a dictionary where each key (an integer)
# corresponds to a body and the value is a list of points.
Initial_Points["BD"]    = {}
Initial_Points["BD"][1] = [[0.75, 0],[-0.75, 0]]

# Forces definition
ForceNames  = ['t','k'] # Define symbolic variables for force points (for example, b1, b2, b3).
ForceSym    = symVars_definition(ForceNames,globals()) # DO NOT MODIFY

# Define the structure for Force.
# Forces applied on body-defined points (PointsBD):
Force["PointsBD"] = []

# Forces applied at the center of gravity (CG):
Force["CG"] = [[1,0,0.1*sym.exp(-t),0]]

# Tension springs: stored as a list of tuples: (connection, [l0, stiffness]).
Force["TensionSpring"] = [(('GR00','BD10'),[5,k]),
                          (('GR01','BD11'),[5,k])
                            ]

# Tension dampers: here we have two pairs.
# The first pair uses a constant damping coefficient.
# The second uses a lambda function to represent a nonlinear damping coefficient.
Force["TensionDamper"] = [(('GR00','BD10'),1),
                          (('GR01','BD11'),1)]

# Torsion springs: here the first element is a list of parameters and the second element is a lambda.
Force["TorsionSpring"] = [((1,2),[0,10]),
                          ((1,3),[0,10])]

# Torsion dampers:
Force["TorsionDamper"] = [((1,2),0),
                          ((1,3),0)]

ForcesPointsSym = PointsSym + ForceSym 

# Initial conditions
# Create the JointSystem using the from_data class method.
joint_system        = JointSystem.from_data(joints, types, parent_cg_to_joint, joint_to_child_cg, prismatic_direction) # DO NOT MODIFY
Q, QD, check, NDOF, Bodycol   = joint_system.coordinate_finder() # DO NOT MODIFY

print(Q)
print(QD)
print(NDOF)
print(Bodycol)

print(joint_system.Q)

[X_1, Z_1, Theta_1, Theta_2, Theta_3]
[XD_1, ZD_1, ThetaD_1, ThetaD_2, ThetaD_3]
[3, 1, 1]
[[0, 1, 2], [3], [4]]


AttributeError: 'JointSystem' object has no attribute 'Q'

In [3]:
# Do not modify 
Initial_Points  = {} 
Force           = {} 
# End of Do not Modify

# Example 3
Reference_frame_Origin = np.array([0,0])

# Bodies definition
dataNames   = [] # Define symbolic variables for force points (for example, b1, b2, b3).
BodyDataSym = symVars_definition(dataNames,globals())


joints              = [[0, 1],[1, 2]] # Joint connectivity: [parent, child]
types               = ['R', 'R']  # Joint types: 'R' for revolute, 'P' for prismatic, 'F' for floating
parent_cg_to_joint  = [[3, 0],[0,1]] # Vectors from parent's center-of-gravity (CG) to the joint location.
joint_to_child_cg   = [[0, 2],[0,4]] # Vectors from the joint to the child's CG.
prismatic_direction = [[np.nan, np.nan],[np.nan, np.nan]] # For prismatic joints, the direction vector; for others, [nan, nan] is used.
prismatic_direction = normalize_prismatic(prismatic_direction)

# Points definition
PointNames  = [] # Define symbolic variables for force points (for example, b1, b2, b3).
PointsSym   = symVars_definition(PointNames,globals())

# Define the structure for initial points.
Initial_Points          = {}
# Ground points: these are fixed and given as [x, z] coordinates.
Initial_Points["GR"]    = []

# Body-defined points ("BD"): we use a dictionary where each key (an integer)
# corresponds to a body and the value is a list of points.
Initial_Points["BD"]    = {}

# Forces definition
ForceNames  = [] # Define symbolic variables for force points (for example, b1, b2, b3).
ForceSym    = symVars_definition(ForceNames,globals())

# Define the structure for Force.
Force = {}
# Forces applied on body-defined points (PointsBD):
Force["PointsBD"] = []

# Forces applied at the center of gravity (CG):
Force["CG"] = []

# Tension springs: stored as a list of tuples: (connection, [l0, stiffness]).
# Here "BD41" means (for example) the 1st point on body 4.
Force["TensionSpring"] = []

# Tension dampers: here we have two pairs.
# The first pair uses a constant damping coefficient.
# The second uses a lambda function to represent a nonlinear damping coefficient.
Force["TensionDamper"] = []

# Torsion springs: here the first element is a list of parameters and the second element is a lambda.
Force["TorsionSpring"] = []

# Torsion dampers:
Force["TorsionDamper"] = []

ForcesPointsSym = PointsSym + ForceSym 

In [5]:
# Initial conditions
# Create the JointSystem using the from_data class method.
joint_system        = JointSystem.from_data(joints, types, parent_cg_to_joint, joint_to_child_cg, prismatic_direction) # DO NOT MODIFY
Q, QD, _, NDOF, Bodycol   = joint_system.coordinate_finder() # DO NOT MODIFY

print(Q)
print(QD)
print(NDOF)
print(Bodycol)

# If you are unsure what are the systems DOF run the example and you will see on screen
ic                  = np.zeros(2*sum(NDOF)) # Multiplied by 2 because is position and velocity
ic[0]               = 0.1  # Initial position of the first joint

# Parameters to loop over
ForcesPointsNum = np.ones(len(ForcesPointsSym))
BodyDataNum     = np.ones(len(BodyDataSym))

# Simulation
TimeStep    = 0.05
tspan       = 5.
g           = 9.81                      # gravity
gVec        = np.ones((len(types),1))   # Percentage of gravity acting on each body
m0          = np.ones(len(types))
J0          = np.ones(len(types))

# Animation
animation_on    = 1 # Display animation:1, 0-otherwise
SaveMovieOn     = None # Extension:gif Saves animation if given a string as name, otherwise set as None
plotTstep       = 5 # how many times faster are we plotting w.r.t to simulation time

[Theta_1, Theta_2]
[ThetaD_1, ThetaD_2]
[1, 1]
[[0], [1]]


In [6]:
#%% Checks section: ensures everything is defined correctly
t = sym.symbols('t')

mb.check.bodies(joints, types, parent_cg_to_joint, joint_to_child_cg, prismatic_direction)
mb.check.points_forces(joints, types, Initial_Points, Force, t)

NBodies = len(joints)

if len(m0) != NBodies or len(J0) != NBodies or not np.all(m0) or not np.all(J0):
    raise ValueError("The mass matrix has zero-valued entries check m0 and J0")
    
'''
The following sections are for user visualization but unnecessary for the
main to run. Uncomment if you want to check how your bodies tables is defines
or if unsure on what the initial conditions are and what is the order
'''
#%% Display tables
# Print the joint system for inspection.
print('\nBodies and joints table:')
joint_system.display_table()

# Display points table
print('\nPoints table:')
mb.tables.PointsTable(Initial_Points)
print('\nForces table:')
mb.tables.ForceTable(Force)

#%% Problem variables and required initial conditions
ic_vars = Q + QD

print('\nThe variables that require initial conditions, in this order, are:\n' + str(ic_vars) + '\n')

Table validation successful:	 No errors found.
Forces validation successful:	 No errors found.
Points validation successful:	 No errors found.


Bodies and joints table:
 Joint Connected Bodies Joint Type ParentCGtoJoint JointtoChildCG Prismatic Direction
     1              0 1          R          [3, 0]         [0, 2]            NaN  NaN
     2              1 2          R          [0, 1]         [0, 4]            NaN  NaN

Points table:
Empty DataFrame
Columns: [Body, PointType, PointID, Rel coordinates]
Index: []

Forces table:
Empty DataFrame
Columns: [ForceType, BodiesConnected, PointType, PointID, ForceComponents, SpringParameters, DamperParameters]
Index: []

The variables that require initial conditions, in this order, are:
[Theta_1, Theta_2, ThetaD_1, ThetaD_2]

